In [1]:
import pandas as pd
import requests
import os
import tweepy
import re
import requests
from bs4 import BeautifulSoup
import urllib

import nltk
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')


from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
model = ResNet50(weights='imagenet')

import numpy as np
import datetime

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Using TensorFlow backend.


102858752/102853048 [==============================] - 5s 0us/step


# 1. Gather Data

####  Load the twitter data in hand through twitter-archive-enhanced.csv
####  Load the dog name prediction data availed in image-predictions.tsv

In [2]:
#load the twitter info available at hand
twitter_archive_enhanced = pd.read_csv("twitter-archive-enhanced.csv")
twitter_archive_cleaned = twitter_archive_enhanced.copy()
twitter_archive_cleaned

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None
5,891087950875897856,NaN,NaN,2017-07-29 00:08:17 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a majestic great white breaching ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891087950...,13,10,None,None,None,None,None
6,890971913173991426,NaN,NaN,2017-07-28 16:27:12 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Jax. He enjoys ice cream so much he gets ...,NaN,NaN,NaN,"https://gofundme.com/ydvmve-surgery-for-jax,ht...",13,10,Jax,None,None,None,None
7,890729181411237888,NaN,NaN,2017-07-28 00:22:40 +0000,"<a href=""http://twitter.com/download/iphone"" r...",When you watch your owner call another dog a g...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890729181...,13,10,None,None,None,None,None
8,890609185150312448,NaN,NaN,2017-07-27 16:25:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Zoey. She doesn't want to be one of th...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890609185...,13,10,Zoey,None,None,None,None
9,890240255349198849,NaN,NaN,2017-07-26 15:59:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Cassie. She is a college pup. Studying...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890240255...,14,10,Cassie,doggo,None,None,None


In [3]:
#download the image-predictions file from Udacity Server only if it does not exists
if not os.path.exists("image-predictions.tsv"):
    with open ("image-predictions.tsv",'wb') as fp:
        response = requests.get("https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv")
        fp.write(response.content)

#load it and visually inspect
image_predictions = pd.read_csv("image-predictions.tsv",delimiter="\t")
image_predictions

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True
5,666050758794694657,https://pbs.twimg.com/media/CT5Jof1WUAEuVxN.jpg,1,Bernese_mountain_dog,0.651137,True,English_springer,0.263788,True,Greater_Swiss_Mountain_dog,0.016199,True
6,666051853826850816,https://pbs.twimg.com/media/CT5KoJ1WoAAJash.jpg,1,box_turtle,0.933012,False,mud_turtle,0.045885,False,terrapin,0.017885,False
7,666055525042405380,https://pbs.twimg.com/media/CT5N9tpXIAAifs1.jpg,1,chow,0.692517,True,Tibetan_mastiff,0.058279,True,fur_coat,0.054449,False
8,666057090499244032,https://pbs.twimg.com/media/CT5PY90WoAAQGLo.jpg,1,shopping_cart,0.962465,False,shopping_basket,0.014594,False,golden_retriever,0.007959,True
9,666058600524156928,https://pbs.twimg.com/media/CT5Qw94XAAA_2dP.jpg,1,miniature_poodle,0.201493,True,komondor,0.192305,True,soft-coated_wheaten_terrier,0.082086,True


# 2. Assess the Data


### Quality isssues:


#### 1. Completeness:
* 1976 entries do not have 'Stage' info.
* 281 entries of dog breed names are missing.

#### 2. Validity:
* Name do not comply to the naming standard; there are 55 entries as "a".
* There are ratings given on reply text message. we are trying find a trend in the root message not the subsequent discussion for the tweet. The reason being, one we do not have access to all reply messages. Two, the study is on weratedogs tweet analysis and not replies :)
* Erraneous datatype - 
    * timestamp, retweeted_status_timestamp (to_datetime), 
    * all "_id" to int64 as integer operations are faster than string. And also makes like easy to do simple logical operator in subset selection.

#### 3. Accuracy:
* p2_* & p3_* are not required as we have p1 > p2 > p3
* Dog breed names also has names other than dog breed names. p1_dog indicates that whether the name identified is dog name or not.
* rating_numerator is not float type and hence 13.5, 11.27, 11.26, 9.75 & 9.5 rating are missing in the data set.

#### 4. Consistency:
* source variable do not have the variables stored along with html tag; like other variable object variables should have the core value and not the residual of the extract.
* Dog breed names are not consistent with respect to usage of capital letters.

### Tidiness Issues:
* Twitter archive and image prediction can be combined to store only dog breed name. 
* Dog stages can be melted to a column and can be considered as category variable


In [4]:
#1.a 1976 entries do not have 'Stage' info.

temp_un = twitter_archive_cleaned[twitter_archive_cleaned.pupper == 'None']
temp_un = temp_un[temp_un.puppo == "None"]
temp_un = temp_un[temp_un.floofer == "None"]
temp_un = temp_un[temp_un.doggo == 'None']
print("In uncleaned dataset: \nUnknown Count :",temp_un.shape[0])
print("Pupper Count :", twitter_archive_cleaned[twitter_archive_cleaned.pupper != 'None'].shape[0])
print("Doggo Count :", twitter_archive_cleaned[twitter_archive_cleaned.doggo != 'None'].shape[0])
print("Floof Count :", twitter_archive_cleaned[twitter_archive_cleaned.floofer != 'None'].shape[0])
print("Puppo Count :", twitter_archive_cleaned[twitter_archive_cleaned.puppo != 'None'].shape[0])

In uncleaned dataset: 
Unknown Count : 1976
Pupper Count : 257
Doggo Count : 97
Floof Count : 10
Puppo Count : 30


In [5]:
#1.b number of entries with dog breed names are missing.
twitter_archive_cleaned = twitter_archive_cleaned.merge(image_predictions[['tweet_id','p1']],how='left')
print("Need dog breed name prediction for :",twitter_archive_cleaned[twitter_archive_cleaned.p1.isnull()].shape[0], " entries")

Need dog breed name prediction for : 281  entries


In [6]:
#2.a Name do not comply to the naming standard; there are 55 entries as "a".
twitter_archive_cleaned.name.value_counts()[:11]

None       745
a           55
Charlie     12
Lucy        11
Cooper      11
Oliver      11
Penny       10
Tucker      10
Lola        10
Bo           9
Winston      9
Name: name, dtype: int64

In [7]:
#2.b there are ratings given on reply text message. we are trying find a trend in the root message not the subsequent 
# discussion for the tweet. The reason being, one we do not have access to all reply messages. Two, the study is on weratedogs 
#tweet analysis and not replies :)
print("Rating discussion on reply or retweets : \n", list(twitter_archive_cleaned[twitter_archive_cleaned.in_reply_to_status_id > 0].text)[2:5])
print("5 High ratings are : ", sorted(list(twitter_archive_cleaned.rating_numerator))[-5:])
print("5 low  ratings are : ", sorted(list(twitter_archive_cleaned.rating_numerator))[:5])

Rating discussion on reply or retweets : 
 ['@RealKentMurphy 14/10 confirmed', '@ComplicitOwl @ShopWeRateDogs &gt;10/10 is reserved for dogs', "@Jack_Septic_Eye I'd need a few more pics to polish a full analysis, but based on the good boy content above I'm leaning towards 12/10"]
5 High ratings are :  [420, 420, 666, 960, 1776]
5 low  ratings are :  [0, 0, 1, 1, 1]


In [8]:
#2.c Erraneous datatype - timestamp, retweeted_status_timestamp (to_datetime), tweet id, in_reply_to_status_id, retweeted_status_id ( as int64)
twitter_archive_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2356 entries, 0 to 2355
Data columns (total 18 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
p1                            20

In [9]:
#3.a p2_* & p3_* are not required as we have p1 > p2 > p3
if (image_predictions[image_predictions.p1_conf < image_predictions.p2_conf].empty):
    print("p1_conf is always greater than p2_conf")
if (image_predictions[image_predictions.p2_conf < image_predictions.p3_conf].empty):
    print("p2_conf is always greater than p3_conf")

p1_conf is always greater than p2_conf
p2_conf is always greater than p3_conf


In [10]:
#let us load the kaggle dog breed name list (120 names) : https://www.kaggle.com/c/dog-breed-identification/data
dog_breed_name_list = []
with open('dog_breed_names.txt','r') as fp:
    for line in fp:
        dog_breed_name_list.append(line[:-1])

print ("Total number of breed names taken from Kaggle :", len(dog_breed_name_list))
dog_breed_name_list = dog_breed_name_list + list(image_predictions[image_predictions.p1_dog == True].p1)

dog_breed_name_list = list(set([dog.lower() for dog in dog_breed_name_list]))
print ("Total number of breed names along with in hand dataset and kaggle : ", len(dog_breed_name_list))
dog_breed_name_list[5:10]

Total number of breed names taken from Kaggle : 120
Total number of breed names along with in hand dataset and kaggle :  122


['dhole', 'borzoi', 'irish_water_spaniel', 'eskimo_dog', 'japanese_spaniel']

In [11]:
#3.b Dog breed names also has other than dog breed names. p1_dog indicates that whether the name identified is dog name or not.
twitter_archive_cleaned[~twitter_archive_cleaned.p1.str.lower().isin(dog_breed_name_list)].p1.value_counts()[:10]


seat_belt      22
web_site       19
teddy          18
tennis_ball     9
doormat         8
tub             7
swing           7
bath_towel      7
hamster         7
Siamese_cat     7
Name: p1, dtype: int64

In [12]:
#3.c rating_numerator is not float type and hence 13.5, 11.27, 9.5 & 9.75 rating are missing in the data set.
list(twitter_archive_cleaned[twitter_archive_cleaned.text.str.contains("[0-9]*[.][0-9]*/")].text)

['This is Bella. She hopes her smile made you smile. If not, she is also offering you her favorite monkey. 13.5/10 https://t.co/qjrljjt948',
 "RT @dog_rates: This is Logan, the Chow who lived. He solemnly swears he's up to lots of good. H*ckin magical af 9.75/10 https://t.co/yBO5wu…",
 "This is Logan, the Chow who lived. He solemnly swears he's up to lots of good. H*ckin magical af 9.75/10 https://t.co/yBO5wuqaPS",
 "This is Sophie. She's a Jubilant Bush Pupper. Super h*ckin rare. Appears at random just to smile at the locals. 11.27/10 would smile back https://t.co/QFaUiIHxHq",
 "This is Finn. He's very nervous for the game. Has a lot of money riding on it.10/10 would attempt to comfort https://t.co/CbtNfecWiT",
 'RT @dog_rates: This... is a Tyrannosaurus rex. We only rate dogs. Please only send in dogs. Thank you ...10/10 https://t.co/zxw8d5g94P',
 'What jokester sent in a pic without a dog in it? This is not @rock_rates. This is @dog_rates. Thank you ...10/10 https://t.co/nDPaYHrtNX'

In [13]:
#4.a source variable do not have the variables stored along with html tag; like other variable object variables should have the core value and not the residual of the extract.
twitter_archive_cleaned.source.value_counts()

<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>     2221
<a href="http://vine.co" rel="nofollow">Vine - Make a Scene</a>                          91
<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>                       33
<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>      11
Name: source, dtype: int64

In [14]:
#4.b Dog breed names are not consistent with respect to usage of capital letters.
temp_df_name = twitter_archive_cleaned[~twitter_archive_cleaned.p1.isnull()]
print ("Number of Dog name having one or more Capital letters : ",len(list(temp_df_name[~temp_df_name.p1.str.islower()].p1)))
print ("Number of Dog name in Lower Case : ",len(list(temp_df_name[temp_df_name.p1.str.islower()].p1)))

Number of Dog name having one or more Capital letters :  940
Number of Dog name in Lower Case :  1135


# Clean the Data

#### Let us first build couple of procedures which would help us later on in our cleaning job

* *get_text_from_url* : provided a list of url, it identifies the twitter url and gets the full text from the tweet.

* *get_rating_from_text* : provided a tweet text message, it identifies rating given in the form of xx.yy/10. Here, xx and yy are numbers.

* *get_dog_stage_from_text* : Given a tweet text message, it tries to identify 'doggo', 'pupper', 'puppo', 'blep', 'snoot' & 'floof' stages as defined by weratedogs.

* *get_name_from_text* : Given a tweet text message, it tries to identify a name based on common patterns in writing tweets and also leverages the nltk.pos_tag to identify Nouns.

* *get_dog_breed_from_local_image* : tries to identify a dog breed name from an image stored locally as 'local-image.jpg'. This procedure has to be called from get_dog_breed_name_from_url.

* *get_dog_breed_name_from_url*: given a list of url, identifies the images in the url. It then iterates over all image to identify a dog image. The dog image should have at least 70% confidence when it was predicted or else the image is ignored and subsequent image is used for prediction.

* *get_dict_from_url* : given the list of urls; identifies the retweet, favorite and reply count for a tweet.

In [15]:
#let the Data gathering begin because we are the good data scientist , brent :)
#let us define the resuable components

def get_text_from_url(url) :

    tweet_text = ""
    twitter_url = ""
    try:
        #print(type(url))
        url_list = url.split(',')

        for url_i in url_list:
            if "twitter.com" in url_i:
                #print("Setting url: ",url_i)
                twitter_url = url_i
                break
        
        #if there are shrinked url or twitter url does not exists; use the available one
        if twitter_url == "":
            twitter_url = url_list[0]
        
        print("twitter_url :", twitter_url)      
        response = requests.get(twitter_url)
        #print("response message got")
        soup = BeautifulSoup(response.content, 'lxml')
        #print("soup cooked")
        
        try:
            tweet_text = \
            soup.find('p', 
                      class_='TweetTextSize TweetTextSize--jumbo js-tweet-text tweet-text').get_text()
            #print("loaded the tweet text")
        except:
            #print("tweet not found....ignored !!!")
            pass
        
    except:
        #no point in going further as we dont have breed of the dog.
        #tweet_text = ""
        #above 2 are risky because we can have dog breed name of dog or dog name which might 
        #handy in our analysis.
        print("Error in url : ", url)
        pass
    print(tweet_text)
    return tweet_text
#test the function: get_text_from_url
print(get_text_from_url('https://t.co/Vm7oRwuRK9'))

twitter_url : https://t.co/Vm7oRwuRK9
This is Badger. Today he embarks on his first plane ride. Already checked a bag containing a single stick. 13/10 safe travels Badgerpic.twitter.com/IVW2W1qgCv
This is Badger. Today he embarks on his first plane ride. Already checked a bag containing a single stick. 13/10 safe travels Badgerpic.twitter.com/IVW2W1qgCv


In [16]:
#let us get the rating out of text
def get_rating_from_text(text):
    #to include his first rating of 8/10
    try:
        rating_str = re.search('[0-9][0-9[0-9]?[0-9]?.?[0-9]?[0-9]?/[0-9][0-9][0-9]',text).group(0)
        numerator_denom = rating_str.split("/")
        return [float(numerator_denom[0]), float(numerator_denom[0])]
    except:
        return [-1.0,-1.0]

#test the function: get_rating_from_text
print(get_rating_from_text(get_text_from_url('https://t.co/Vm7oRwuRK9')))

twitter_url : https://t.co/Vm7oRwuRK9
This is Badger. Today he embarks on his first plane ride. Already checked a bag containing a single stick. 13/10 safe travels Badgerpic.twitter.com/IVW2W1qgCv
[-1.0, -1.0]


In [17]:
#let us search the dog stage from the text
def get_dog_stage_from_text(text):
    dog_stages = ['doggo', 'pupper', 'puppo', 'blep', 'snoot', 'floof']

    stage = ""
    for word in nltk.word_tokenize(text):
        for s in dog_stages:
            #print(s,word)
            if s in word.lower():
                stage = s
                break
        if len(stage) > 0:
            break

    #print("stage:",stage)
    return stage


#test the function: get_dog_stage_from_text
print(get_dog_stage_from_text(get_text_from_url('https://t.co/Vm7oRwuRK9')))
print(get_dog_stage_from_text(get_text_from_url('https://twitter.com/dog_rates/status/1014563241572397061')))
print(get_dog_stage_from_text(get_text_from_url('https://t.co/r28jFx9uyF')))

twitter_url : https://t.co/Vm7oRwuRK9
This is Badger. Today he embarks on his first plane ride. Already checked a bag containing a single stick. 13/10 safe travels Badgerpic.twitter.com/IVW2W1qgCv

twitter_url : https://twitter.com/dog_rates/status/1014563241572397061
This is Misty. She accidentally dropped her ball in the pool. Not a problem. 14/10 for the deep sea diver doggopic.twitter.com/kookhestmb
doggo
twitter_url : https://t.co/r28jFx9uyF
This is Walter. He won't start hydrotherapy without his favorite floatie. 14/10 keep it pup Walterpic.twitter.com/r28jFx9uyF



In [18]:
# Identify a name in the tweet text
def get_name_from_text(text):
    dog_stages = ['doggo', 'pupper', 'puppo', 'blep', 'snoot', 'floof']
    nouns_tag = ['NN','NNS','NNP','NNPS']
    dog_stopwords = stopwords.words('english')
    custom_stopwards = ['please','thank', 'rt','pupdate','everyone','prefers','V-DAY','everybody','guys','Christmas','dogs',
                        'people'
                        ]
    for w in text.split(" "):
        if "@" in w or "#" in w:
            custom_stopwards.append(w[1:])
    custom_stopwards = [x.lower() for x in custom_stopwards]
    
    dog_stopwords = dog_stopwords + custom_stopwards
    name = ""
   
    #now check if the sentense starts with This is X. or Meet X. remove RT
    first_sent = nltk.sent_tokenize(text)[0]
    #print("first sentence :", first_sent)
    first_sent1 = first_sent.split(":")
    if len(first_sent1) > 1:
        first_sent = first_sent1[1][1:]
    else:
        first_sent = first_sent1[0]    
    #print("first sentence now:", first_sent,"\npost split: ", first_sent.split(" "))
    if len(first_sent.split(" ")) == 3:
        if first_sent.split(" ")[0] == "This":
            if first_sent.split(" ")[1] == "is":
                name = first_sent.split(" ")[2][:-1]
    elif len(text.split("Meet ")) > 1:
        name = text.split("Meet ")[1].split(".")[0]
    elif len(text.split("Say hello to ")) > 1:
        name = text.split("Say hello to ")[1].split(".")[0]
    else:
        if len(text.split(":")) > 2 :
            text = text.split(":")[1]
        elif len(text.split("https:")) > 1 :
            text = text.split(":")[0]
            
        tagged = nltk.pos_tag(nltk.word_tokenize(text))
        #print(len(tagged))
        tagged_lower = nltk.pos_tag(nltk.word_tokenize(text.lower()))
        #print(len(tagged_lower))
        if len(tagged) == len(tagged_lower):
            for iterate in range(len(tagged)):
                if tagged_lower[iterate][0] not in dog_stopwords:
                    #print(tagged[iterate],tagged_lower[iterate])
                    #if tag[1] == 'NNP' and tag[0].lower() not in stopwords.words('english'):
                    if tagged[iterate][1] in nouns_tag and \
                    tagged_lower[iterate][1] in nouns_tag and \
                    tagged[iterate][0] != tagged_lower[iterate][0]:
                        name = tagged[iterate][0]
                        break
        else:
            for iterate in range(len(tagged)):
                if tagged[iterate][0].lower() not in dog_stopwords:
                    if tagged[iterate][1] in nouns_tag:
                        name = tagged[iterate][0]
                        break

    print("Tweet Message : ", text, "\nDog Name : ", name)
   
    return name

#test get_name_from_text
sample_text = '''RT @dog_rates: This is a Emmy. She was adopted today. Massive round of pupplause for Emmy and her new family. 14/10 for all involved https://…
'''
print(get_name_from_text(sample_text))
sample_text = '''"This is an odd dog. Hard on the outside but loving on the inside. Petting still fun. Doesn't play catch well. 2/10 https://t.co/v5A4vzSDdc'''
print(get_name_from_text(sample_text))
sample_text = '''This is a purebred Piers Morgan. pupper Loves to Netflix and chill. Always looks like he forgot to unplug the iron. 6/10 https://t.co/DWnyCjf2mx'''
print(get_name_from_text(sample_text))
sample_text = '''Please only send dogs. We don't rate mechanics, no matter how h*ckin good. Thank you... 13/10 would sneak a pat https://t.co/Se5fZ9wp5E'''
print(get_name_from_text(sample_text))
sample_text = '''I couldn't make it to the #WKCDogShow BUT I have people there on the ground relaying me the finest pupper pics possible. 13/10 for all https://t.co/jd6lYhfdH'''
print(get_name_from_text(sample_text))

Tweet Message :   This is a Emmy. She was adopted today. Massive round of pupplause for Emmy and her new family. 14/10 for all involved https 
Dog Name :  Emmy
Emmy
Tweet Message :  "This is an odd dog. Hard on the outside but loving on the inside. Petting still fun. Doesn't play catch well. 2/10 https 
Dog Name :  

Tweet Message :  This is a purebred Piers Morgan. pupper Loves to Netflix and chill. Always looks like he forgot to unplug the iron. 6/10 https 
Dog Name :  Piers
Piers
Tweet Message :  Please only send dogs. We don't rate mechanics, no matter how h*ckin good. Thank you... 13/10 would sneak a pat https 
Dog Name :  

Tweet Message :  I couldn't make it to the #WKCDogShow BUT I have people there on the ground relaying me the finest pupper pics possible. 13/10 for all https 
Dog Name :  



In [19]:
#let us get the last item: dog name
def get_dog_breed_from_local_image():
    img_path = r'local-image.jpg'
    if os.path.exists(img_path):
        img = image.load_img(img_path, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        preds = model.predict(x)
        # decode the results into a list of tuples (class, description, probability)
        # (one such list for each sample in the batch)
        #print("Prediction Done !!",decode_predictions(preds, top=3)[0])
        if decode_predictions(preds, top=3)[0][0][2] > 0.7 and \
           decode_predictions(preds, top=3)[0][0][1].lower() in dog_breed_name_list:
                return decode_predictions(preds, top=3)[0][0][1]
    return ""

#test the function: get_dog_breed_from_local_image
get_text_from_url('https://t.co/Vm7oRwuRK9')
print(get_dog_breed_from_local_image())
get_text_from_url('https://twitter.com/dog_rates/status/1014563241572397061')
print(get_dog_breed_from_local_image())

twitter_url : https://t.co/Vm7oRwuRK9
This is Badger. Today he embarks on his first plane ride. Already checked a bag containing a single stick. 13/10 safe travels Badgerpic.twitter.com/IVW2W1qgCv
40960/35363 [==================================] - 0s 2us/step
Labrador_retriever
twitter_url : https://twitter.com/dog_rates/status/1014563241572397061
This is Misty. She accidentally dropped her ball in the pool. Not a problem. 14/10 for the deep sea diver doggopic.twitter.com/kookhestmb
Labrador_retriever


In [20]:
def get_images_from_url(url):
    images_url_list = []
    try:
        for u in url.split(","):
            response = requests.get(u)
            soup = BeautifulSoup(response.content, 'lxml')
            try:
                image_tag_list = \
                    soup.find('div', class_="AdaptiveMedia-container").find_all('img')
                #image_loc = []
                for igl in image_tag_list:
                    images_url_list.append(igl['src'])
                #print(image_loc)
            except:
                pass
    except:
        print("Url has issue : ", url)

    return images_url_list

get_images_from_url('https://twitter.com/dog_rates/status/879862464715927552/photo/1')

['https://pbs.twimg.com/media/DDXmPreXUAA3QGJ.jpg',
 'https://pbs.twimg.com/media/DDXmPrrXoAI42eo.jpg',
 'https://pbs.twimg.com/media/DDXmPrbWAAEKMvy.jpg']

In [21]:
#define a function to know the dog breed name from the url
def get_dog_breed_name_from_url(url):
    images_list = get_images_from_url(url)
    #print(images_list)
    for image in images_list:
        #let us also foresee if breed recognition through image is feasible
        if os.path.exists("local-image.jpg"):
            os.remove("local-image.jpg")
            
        try:
            urllib.request.urlretrieve(image, "local-image.jpg")

            dog_bread_name = get_dog_breed_from_local_image()
            #print(dog_bread_name)
            if  dog_bread_name != "":
                return dog_bread_name
        except:
            pass            
    
    return ""

#test get_dog_breed_name_from_url
get_dog_breed_name_from_url('https://t.co/Vm7oRwuRK9')

'Labrador_retriever'

In [22]:
#let us load the reply count, retweet count and favorite count
def get_dict_from_url(url) :
    
    dict_out = {}
    dict_out['retweet_count'] = dict_out['favorite_count'] = dict_out['reply_count'] = 0
    
    try:
        for url in url.split(','):
            try:
                response = requests.get(url)
                #print("response message got")
                soup = BeautifulSoup(response.content, 'lxml')
                #print("soup cooked")

                dict_out['retweet_count'] = \
                soup.find('li', 
                      class_='js-stat-count js-stat-retweets stat-count').find('a')['data-tweet-stat-count']

                dict_out['favorite_count'] = \
                soup.find('li', 
                      class_='js-stat-count js-stat-favorites stat-count').find('a')['data-tweet-stat-count']

                dict_out['reply_count'] = 0
                dict_out['reply_count'] = \
                    soup.find('span', 
                      class_='ProfileTweet-actionCountForPresentation').get_text()   
                if dict_out['reply_count'] == '':
                    #print("Fetching reply count Again")
                    dict_out['reply_count'] = \
                        soup.find('span', 
                          class_='ProfileTweet-actionCount')['data-tweet-stat-count']
                
                if dict_out['reply_count'] > 0 or dict_out['favorite_count'] > 0 or dict_out['retweet_count'] > 0:
                    break

                    #print("Fetched :",dict_out['reply_count'])
            except:
                pass

    except:
        print("Something went wrong ; returning empty dict:",url)
        return [0,0,0]

    #print(dict_out)    
    return [dict_out['retweet_count'], dict_out['favorite_count'], dict_out['reply_count']]

#test get_dict_from_url
print(get_dict_from_url('https://twitter.com/dog_rates/status/709901256215666688/photo/1'))
print(get_dict_from_url('https://twitter.com/dog_rates/status/888078434458587136/photo/1'))
print(get_dict_from_url('https://twitter.com/dog_rates/status/879862464715927552/photo/1'))

['108', '714', '30']
['3485', '21661', '111']
['3504', '22230', '78']


#### Let us first fix the following 2 observations
* 1.a 1976 entries do not have 'Stage' info.
* Dog stages can be melted to a column and can be considered as category variable

##### Define: Extract the stages from the text and check if the all the stages are there in the tweets we have.

##### Code:

In [23]:
twitter_archive_cleaned["stage"] = twitter_archive_cleaned.text.apply(lambda x:get_dog_stage_from_text(x))
twitter_archive_cleaned = twitter_archive_cleaned.drop('doggo', 1)
twitter_archive_cleaned = twitter_archive_cleaned.drop('floofer', 1)
twitter_archive_cleaned = twitter_archive_cleaned.drop('pupper', 1)
twitter_archive_cleaned = twitter_archive_cleaned.drop('puppo', 1)

##### Test:

In [24]:
twitter_archive_cleaned.stage.value_counts()

          1902
pupper     275
doggo      100
floof       38
puppo       38
blep         3
Name: stage, dtype: int64

##### The results are better than the initial data. Now we have less than 1976 missing entries for dog stage.

#### Let us now fix the dog breed name related issues
* 1.b 281 entries of dog breed names are missing.
* 3.b Dog breed names also has other than dog breed names. p1_dog indicates that whether the name identified is dog name or not.
* 4.b Dog breed names are not consistent with respect to usage of capital letters.
* Twitter archive and image prediction can be combined to store only dog breed name. 

##### Define: 
* left join  twitter_archive_cleaned with image_predictions because image_predictions has less samples. Take only p1 because p1_conf > p2_conf > p3_conf
* join twitter_archive_cleaned with image_recognition_keras ( which has image prediction of a dog for each tweet with a confidence more than 70%). Take only image_recognition_keras 'breed_name' column as we have rest of the column are same as that of twitter_archive_cleaned.
* remove non dog breed name from the p1
* replace p1 with breed_name value where p1 = null
* replace p1 with lower case breed names


In [25]:
#check if image recognition by keras model does the better job

if os.path.exists('twitter_archive_cleaned_images.csv'):
    image_recognition_keras = pd.read_csv('twitter_archive_cleaned_images.csv')
else:
    twitter_archive_cleaned['breed_name'] = twitter_archive_cleaned.expanded_urls.apply(lambda x: get_dog_breed_name_from_url(x))
    twitter_archive_cleaned.to_csv('twitter_archive_cleaned_images.csv')
    #twitter_archive_cleaned['re_fav_reply'] = twitter_archive_cleaned.expanded_urls.apply(lambda x: get_dict_from_url(x))
    #twitter_archive_cleaned.to_csv('twitter_archive_cleaned_images.csv')
    image_recognition_keras = twitter_archive_cleaned.copy()
    
image_recognition_keras.breed_name.value_counts()

golden_retriever                  102
Labrador_retriever                 48
Pembroke                           45
Samoyed                            32
pug                                32
Chihuahua                          22
French_bulldog                     17
Pomeranian                         16
malamute                           13
chow                               12
toy_poodle                         11
German_shepherd                    10
cocker_spaniel                      9
American_Staffordshire_terrier      9
Maltese_dog                         7
Bernese_mountain_dog                7
Shetland_sheepdog                   7
Old_English_sheepdog                7
basset                              7
Blenheim_spaniel                    7
dalmatian                           7
Brittany_spaniel                    7
Pekinese                            6
Yorkshire_terrier                   6
Chesapeake_Bay_retriever            6
beagle                              6
miniature_pi

In [26]:
#let us now check how may dogs are there in 2 dfs
#image_df = pd.DataFrame(image_predictions[image_predictions.p1.isin(dog_breed_name_list)])
image_df = image_predictions.copy()
image_df['p1'] = image_df.p1.apply(lambda x: x.lower())
image_df = image_df[image_df.p1.isin(dog_breed_name_list)]
print("available data set has : ", image_df.shape[0], " dog breed names")
image_df = image_df[image_df.p1_conf > 0.7].p1.value_counts().reset_index()
print("available data set has : ", image_df.shape[0], " dog breed names with greater than 70% confidence")
                        
image_df = image_df.sort_values('index').reset_index(drop=True)
#image_df
image_df2 = image_recognition_keras.copy()
image_df2 = image_df2[~image_df2['breed_name'].isnull()]
image_df2['breed_name'] = image_df2['breed_name'].apply(lambda x: x.lower())
image_df2 = image_df2[image_df2.breed_name.isin(dog_breed_name_list)].breed_name.value_counts().reset_index()
#image_df2 = pd.DataFrame(image_recognition_keras[image_recognition_keras.breed_name.isin(dog_breed_name_list)].breed_name.value_counts().reset_index())
image_df2 = image_df2.sort_values('index').reset_index(drop=True)
print("prediction algo has : ", image_df2.shape[0], " dog breed names with greater than 70% confidence")
#image_df2
image_df3 = image_df.merge( image_df2, how="left")
image_df3.columns = [['breed name', 'available count','predicted count']]
image_df3
#Seems like there can be improvement to custom made function; mostly related to how the image is accessed in case of multiple url

available data set has :  1543  dog breed names
available data set has :  86  dog breed names with greater than 70% confidence
prediction algo has :  84  dog breed names with greater than 70% confidence


,breed name,available count,predicted count
0,afghan_hound,1,2.0
1,african_hunting_dog,1,1.0
2,airedale,4,NaN
3,american_staffordshire_terrier,5,9.0
4,basenji,3,4.0
5,basset,6,7.0
6,beagle,5,6.0
7,bernese_mountain_dog,7,7.0
8,black-and-tan_coonhound,1,NaN
9,blenheim_spaniel,6,7.0


##### The above comparison indicates that the keras model performs almost same as that of available data.  Let us now put the predicted values to missing data in the original dataset

In [27]:
twitter_archive_cleaned= twitter_archive_cleaned.merge(image_recognition_keras[['tweet_id', 'breed_name']], how='left')

not_null_entries_before = twitter_archive_cleaned[twitter_archive_cleaned.p1.isnull()].shape[0]
twitter_archive_cleaned['p1'] = twitter_archive_cleaned.p1.apply(lambda x: x if type(x) == type('str') and x.lower() in dog_breed_name_list else None)
valid_breed_name_before = twitter_archive_cleaned[~twitter_archive_cleaned['p1'].isnull()].shape[0]
twitter_archive_cleaned['p1'] = twitter_archive_cleaned.apply( lambda row: row['p1'] if (type(row['p1']) != type(1.0) and type(row['p1']) != type(None)) else row['breed_name'], axis =1 )

valid_breed_name_after = twitter_archive_cleaned[~twitter_archive_cleaned['p1'].isnull()].shape[0]
not_null_entries_after = twitter_archive_cleaned[twitter_archive_cleaned.p1.isnull()].shape[0]

per_change_null = round((not_null_entries_after - not_null_entries_before)/not_null_entries_before,2)
per_change_valid = round((valid_breed_name_after - valid_breed_name_before)/valid_breed_name_before,2)

twitter_archive_cleaned['p1'] = twitter_archive_cleaned['p1'].apply(lambda x : None if type(x) != type('str') else x.lower())
twitter_archive_cleaned = twitter_archive_cleaned.drop('breed_name', 1)
twitter_archive_cleaned = twitter_archive_cleaned.rename(index=str, columns={"p1": "breed_name"})

##### Test:

In [28]:
print("There is ", per_change_null, "% increment is missing values")
print("There is ", per_change_valid, "% increment is valid values")
tt = twitter_archive_cleaned[~twitter_archive_cleaned['breed_name'].isnull()]
print("Breed names with upper case names : ", tt[~tt['breed_name'].str.islower()].shape[0])

There is  1.78 % increment is missing values
There is  0.02 % increment is valid values
Breed names with upper case names :  0


In [29]:
twitter_archive_cleaned

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,breed_name,stage
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,chihuahua,
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,chihuahua,
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,basset,
5,891087950875897856,NaN,NaN,2017-07-29 00:08:17 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a majestic great white breaching ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891087950...,13,10,None,chesapeake_bay_retriever,
6,890971913173991426,NaN,NaN,2017-07-28 16:27:12 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Jax. He enjoys ice cream so much he gets ...,NaN,NaN,NaN,"https://gofundme.com/ydvmve-surgery-for-jax,ht...",13,10,Jax,appenzeller,
7,890729181411237888,NaN,NaN,2017-07-28 00:22:40 +0000,"<a href=""http://twitter.com/download/iphone"" r...",When you watch your owner call another dog a g...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890729181...,13,10,None,pomeranian,
8,890609185150312448,NaN,NaN,2017-07-27 16:25:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Zoey. She doesn't want to be one of th...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890609185...,13,10,Zoey,irish_terrier,
9,890240255349198849,NaN,NaN,2017-07-26 15:59:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Cassie. She is a college pup. Studying...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890240255...,14,10,Cassie,pembroke,doggo


In [30]:
twitter_archive_cleaned.breed_name.value_counts()

golden_retriever                  159
labrador_retriever                101
pembroke                           91
chihuahua                          83
pug                                57
samoyed                            46
chow                               45
toy_poodle                         40
pomeranian                         39
cocker_spaniel                     31
malamute                           30
french_bulldog                     29
chesapeake_bay_retriever           24
miniature_pinscher                 23
siberian_husky                     20
german_shepherd                    20
staffordshire_bullterrier          20
cardigan                           19
eskimo_dog                         19
beagle                             18
maltese_dog                        18
shetland_sheepdog                  18
lakeland_terrier                   17
rottweiler                         17
italian_greyhound                  17
shih-tzu                           17
kuvasz      

#### 2.a Name do not comply to the naming standard; there are 55 entries as "a".

##### Define: 
* remove stopwords from dog names
* remove any lower case name

##### Code:

In [31]:
twitter_archive_cleaned.name = twitter_archive_cleaned.name.apply( lambda x : x if x.lower() not in stopwords.words('english') else 'None')
#tt = twitter_archive_cleaned[~twitter_archive_cleaned.name.isnull()]
twitter_archive_cleaned.loc[twitter_archive_cleaned.name.str.islower(),'name'] = 'None'

##### Test:

In [32]:
twitter_archive_cleaned.name.value_counts()

None         855
Charlie       12
Cooper        11
Oliver        11
Lucy          11
Tucker        10
Lola          10
Penny         10
Winston        9
Bo             9
Sadie          8
Bailey         7
Buddy          7
Toby           7
Daisy          7
Scout          6
Dave           6
Oscar          6
Koda           6
Milo           6
Bella          6
Jack           6
Stanley        6
Jax            6
Rusty          6
Leo            6
Oakley         5
Chester        5
Louis          5
Sunny          5
            ... 
Nigel          1
Leonard        1
Daniel         1
Ralphson       1
Sid            1
Corey          1
Franq          1
Terrenth       1
Reagan         1
Aiden          1
Smiley         1
Durg           1
Grey           1
Rooney         1
Amélie         1
Tove           1
Jeffri         1
Sojourner      1
Godi           1
Craig          1
Snicku         1
Rambo          1
Beebop         1
Zoe            1
Blakely        1
Lassie         1
Iroh           1
Ike           

In [33]:
twitter_archive_cleaned.sample(10).name

830       Jesse
1902       None
1981       Chet
980        Lucy
557       Sonny
2222       None
186        None
932     Charlie
26         Maya
318        None
Name: name, dtype: object

#### 2.b Erraneous datatype - 
    * timestamp, retweeted_status_timestamp (to_datetime), 
    * all "_id" to int64 as integer operations are faster than string. And also makes like easy to do simple logical operator in subset selection
#### 3.c rating_numerator is not float type and hence 13.5, 11.27, 9.5 & 9.75 rating are missing in the data set.

##### Define: All _id variants to int; timestamp to datetime

##### Code:

In [34]:
if os.path.exists("twitter_archive_cleaned_last.csv"):
    twitter_archive_cleaned = pd.read_csv("twitter_archive_cleaned_last.csv")
    twitter_archive_cleaned = twitter_archive_cleaned.drop('Unnamed: 0', 1)
else:
    twitter_archive_cleaned.to_csv("twitter_archive_cleaned_last.csv")

twitter_archive_cleaned.in_reply_to_status_id = pd.to_numeric(twitter_archive_cleaned.in_reply_to_status_id, errors='coerce')\
                                                .fillna(0).astype(np.int64)
twitter_archive_cleaned.in_reply_to_user_id = pd.to_numeric(twitter_archive_cleaned.in_reply_to_user_id, errors='coerce')\
                                                .fillna(0).astype(np.int64) 
twitter_archive_cleaned.in_reply_to_user_id = pd.to_numeric(twitter_archive_cleaned.in_reply_to_user_id, errors='coerce')\
                                                .fillna(0).astype(np.int64)
twitter_archive_cleaned.retweeted_status_id = pd.to_numeric(twitter_archive_cleaned.retweeted_status_id, errors='coerce')\
                                                .fillna(0).astype(np.int64) 
twitter_archive_cleaned.retweeted_status_user_id = pd.to_numeric(twitter_archive_cleaned.retweeted_status_user_id, errors='coerce')\
                                                .fillna(0).astype(np.int64)    

twitter_archive_cleaned.rating_numerator = pd.to_numeric(twitter_archive_cleaned.rating_numerator, errors='coerce')\
                                                .fillna(0).astype(np.float64) 
    
twitter_archive_cleaned.timestamp = pd.to_datetime(twitter_archive_cleaned.timestamp)
twitter_archive_cleaned.retweeted_status_timestamp = pd.to_datetime(twitter_archive_cleaned.retweeted_status_timestamp)
twitter_archive_cleaned = twitter_archive_cleaned.sort_values('timestamp')

##### Test

In [35]:
twitter_archive_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2356 entries, 2355 to 0
Data columns (total 15 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         2356 non-null int64
in_reply_to_user_id           2356 non-null int64
timestamp                     2356 non-null datetime64[ns]
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           2356 non-null int64
retweeted_status_user_id      2356 non-null int64
retweeted_status_timestamp    181 non-null datetime64[ns]
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null float64
rating_denominator            2356 non-null int64
name                          2356 non-null object
breed_name                    1576 non-null object
stage                         454 non-null object
dtypes: datetime64[ns](2), float64(1), int64(6), object(6)
memory usage: 294.5+ KB


In [36]:
twitter_archive_cleaned.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,breed_name,stage
2355,666020888022790149,0,0,2015-11-15 22:32:08,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a Japanese Irish Setter. Lost eye...,0,0,NaT,https://twitter.com/dog_rates/status/666020888...,8.0,10,None,welsh_springer_spaniel,NaN
2354,666029285002620928,0,0,2015-11-15 23:05:30,"<a href=""http://twitter.com/download/iphone"" r...",This is a western brown Mitsubishi terrier. Up...,0,0,NaT,https://twitter.com/dog_rates/status/666029285...,7.0,10,None,redbone,NaN
2353,666033412701032449,0,0,2015-11-15 23:21:54,"<a href=""http://twitter.com/download/iphone"" r...",Here is a very happy pup. Big fan of well-main...,0,0,NaT,https://twitter.com/dog_rates/status/666033412...,9.0,10,None,german_shepherd,NaN
2352,666044226329800704,0,0,2015-11-16 00:04:52,"<a href=""http://twitter.com/download/iphone"" r...",This is a purebred Piers Morgan. Loves to Netf...,0,0,NaT,https://twitter.com/dog_rates/status/666044226...,6.0,10,None,rhodesian_ridgeback,NaN
2351,666049248165822465,0,0,2015-11-16 00:24:50,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a 1949 1st generation vulpix. Enj...,0,0,NaT,https://twitter.com/dog_rates/status/666049248...,5.0,10,None,miniature_pinscher,NaN


#### 2.b  there are ratings given on reply text message. we are trying find a trend in the root message not the subsequent discussion for the tweet. The reason being, one we do not have access to all reply messages. Two, the study is on weratedogs tweet analysis and not replies :)

##### Define: remove the reply messages from the data sets.

##### Code:

In [37]:
if os.path.exists("twitter_archive_cleaned_last_1.csv"):
    twitter_archive_cleaned = pd.read_csv("twitter_archive_cleaned_last_1.csv")
    twitter_archive_cleaned = twitter_archive_cleaned.drop('Unnamed: 0', 1)
else:
    twitter_archive_cleaned.to_csv("twitter_archive_cleaned_last_1.csv")

#Let us first work on replies
list_reply_id = list(twitter_archive_cleaned[twitter_archive_cleaned.in_reply_to_status_id > 0].in_reply_to_status_id)
print ("How many reply messages are there totally ?\nAns : ", len(list_reply_id))
print("How many of the original messages are present in the dataset for which a reply is also present ? \nAns: ", \
     twitter_archive_cleaned[twitter_archive_cleaned.tweet_id.isin(list_reply_id)].shape[0])

list_orig_present = list(twitter_archive_cleaned[twitter_archive_cleaned.tweet_id.isin(list_reply_id)].tweet_id)
list_retweet_id = list(twitter_archive_cleaned[twitter_archive_cleaned.retweeted_status_id > 0].retweeted_status_id)
twitter_archive_cleaned = twitter_archive_cleaned[~twitter_archive_cleaned.in_reply_to_status_id.isin(list_orig_present)]


#Let us work on retweets
print("How many retweets are there?\nAns : ", len(list_retweet_id))

list_orig_present = list(twitter_archive_cleaned[twitter_archive_cleaned.tweet_id.isin(list_retweet_id)].tweet_id)
twitter_archive_cleaned = twitter_archive_cleaned[~twitter_archive_cleaned.retweeted_status_id.isin(list_orig_present)]
print("For each retweet, are the original message present in the sample if yes how many ?\nAns: ",len(list_orig_present) )

How many reply messages are there totally ?
Ans :  78
How many of the original messages are present in the dataset for which a reply is also present ? 
Ans:  33
How many retweets are there?
Ans :  181
For each retweet, are the original message present in the sample if yes how many ?
Ans:  112


In [38]:
print (list(twitter_archive_cleaned[twitter_archive_cleaned.text.str.contains("[0-9]*[.][0-9]*/")]['text']))
twitter_archive_cleaned[twitter_archive_cleaned.text.str.contains("[0-9]*[.][0-9]*/")][['text','tweet_id']]


['Here we have uncovered an entire battalion of holiday puppers. Average of 11.26/10 https://t.co/eNm2S6p9BD', '"FOR THE LAST TIME I DON\'T WANNA PLAY TWISTER ALL THE SPOTS ARE GREY DAMN IT CINDY" ...10/10 https://t.co/uhQNehTpIu', 'For the last time, WE. DO. NOT. RATE. BULBASAUR. We only rate dogs. Please only send dogs. Thank you ...9/10 https://t.co/GboDG8WhJG', 'Please stop sending in saber-toothed tigers. This is getting ridiculous. We only rate dogs.\n...8/10 https://t.co/iAeQNueou8', "I know it's tempting, but please stop sending in pics of Donald Trump. Thank you ...9/10 https://t.co/y35Y1TJERY", '"Yes hi could I get a number 4 with no pickles" ...12/10 https://t.co/kQPVxqA3gq', "This is Layla. She's giving you a standing ovation.13/10 just magnificent (vid by @CSBrzezinski) https://t.co/KxYXHUHUi2", "When you're just relaxin and having a swell time but then remember you have to fill out the FAFSA ...11/10 https://t.co/qy33OBcexg", '*lets out a tiny whimper and then collapses* 

,text,tweet_id
643,Here we have uncovered an entire battalion of ...,680494726643068929
728,"""FOR THE LAST TIME I DON'T WANNA PLAY TWISTER ...",684594889858887680
745,"For the last time, WE. DO. NOT. RATE. BULBASAU...",685532292383666176
920,Please stop sending in saber-toothed tigers. T...,697259378236399616
983,"I know it's tempting, but please stop sending ...",702217446468493312
1014,"""Yes hi could I get a number 4 with no pickles...",704847917308362754
1029,This is Layla. She's giving you a standing ova...,706153300320784384
1033,When you're just relaxin and having a swell ti...,706291001778950144
1089,*lets out a tiny whimper and then collapses* ....,709852847387627521
1244,"""Ello this is dog how may I assist"" ...10/10 h...",733482008106668032


In [39]:
print(list(twitter_archive_cleaned[twitter_archive_cleaned['tweet_id'] == 883482846933004288].text))
twitter_archive_cleaned.loc[twitter_archive_cleaned['tweet_id'] == 883482846933004288,'rating_numerator'] = 13.5

print(list(twitter_archive_cleaned[twitter_archive_cleaned['tweet_id'] == 786709082849828864].text))
twitter_archive_cleaned.loc[twitter_archive_cleaned['tweet_id'] == 786709082849828864,'rating_numerator'] = 9.75

print(list(twitter_archive_cleaned[twitter_archive_cleaned['tweet_id'] == 778027034220126208].text))
twitter_archive_cleaned.loc[twitter_archive_cleaned['tweet_id'] == 778027034220126208,'rating_numerator'] = 11.27

print(list(twitter_archive_cleaned[twitter_archive_cleaned['tweet_id'] == 680494726643068929].text))
twitter_archive_cleaned.loc[twitter_archive_cleaned['tweet_id'] == 680494726643068929,'rating_numerator'] = 11.26

print("Correcting rating after manually going through tweet text : ", list(twitter_archive_cleaned[twitter_archive_cleaned.rating_numerator == 17].text))
twitter_archive_cleaned.loc[twitter_archive_cleaned.rating_numerator == 17, 'rating_numerator'] = 13

print("Correcting rating after manually going through tweet text : ", list(twitter_archive_cleaned[twitter_archive_cleaned.tweet_id == 835246439529840640].text))
twitter_archive_cleaned.loc[twitter_archive_cleaned.tweet_id == 835246439529840640, 'rating_numerator'] = 13
twitter_archive_cleaned.loc[twitter_archive_cleaned.tweet_id == 835246439529840640, 'rating_denominator'] = 10

#invalid tweet & hence remove it
twitter_archive_cleaned = twitter_archive_cleaned[twitter_archive_cleaned.tweet_id != 855862651834028034]

['This is Bella. She hopes her smile made you smile. If not, she is also offering you her favorite monkey. 13.5/10 https://t.co/qjrljjt948']
["This is Logan, the Chow who lived. He solemnly swears he's up to lots of good. H*ckin magical af 9.75/10 https://t.co/yBO5wuqaPS"]
["This is Sophie. She's a Jubilant Bush Pupper. Super h*ckin rare. Appears at random just to smile at the locals. 11.27/10 would smile back https://t.co/QFaUiIHxHq"]
['Here we have uncovered an entire battalion of holiday puppers. Average of 11.26/10 https://t.co/eNm2S6p9BD']
Correcting rating after manually going through tweet text :  ['@roushfenway These are good dogs but 17/10 is an emotional impulse rating. More like 13/10s']
Correcting rating after manually going through tweet text :  ["@jonnysun @Lin_Manuel ok jomny I know you're excited but 960/00 isn't a valid rating, 13/10 is tho"]


##### Test:

In [40]:
twitter_archive_cleaned.rating_numerator.value_counts()

12.00      515
10.00      442
11.00      434
13.00      321
9.00       155
8.00        98
7.00        54
14.00       48
5.00        35
6.00        32
3.00        19
4.00        16
2.00         9
1.00         6
15.00        2
0.00         2
60.00        1
88.00        1
144.00       1
44.00        1
143.00       1
182.00       1
666.00       1
99.00        1
165.00       1
9.75         1
204.00       1
121.00       1
45.00        1
1776.00      1
50.00        1
420.00       1
11.27        1
11.26        1
13.50        1
84.00        1
24.00        1
80.00        1
Name: rating_numerator, dtype: int64

##### 4.a source variable do not have the variables stored along with html tag; like other variable object variables should have the core value and not the residual of the extract.

##### Define: remove html tags from it 

##### Code:

In [41]:
twitter_archive_cleaned.source = twitter_archive_cleaned.source.apply(lambda x: x.split('>')[1].split('<')[0])

##### Test

In [42]:
twitter_archive_cleaned.source.value_counts()

Twitter for iPhone     2075
Vine - Make a Scene      91
Twitter Web Client       33
TweetDeck                11
Name: source, dtype: int64

In [43]:
if os.path.exists("twitter_archive_cleaned_last_2.csv"):
    twitter_archive_cleaned = pd.read_csv("twitter_archive_cleaned_last_2.csv")
    twitter_archive_cleaned = twitter_archive_cleaned.drop('Unnamed: 0', 1)
else:
    twitter_archive_cleaned['re_fav_reply'] = twitter_archive_cleaned.tweet_id.apply(lambda x: get_dict_from_url('https://twitter.com/dog_rates/status/'+str(x)))
    twitter_archive_cleaned.to_csv("twitter_archive_cleaned_last_2.csv")

twitter_archive_cleaned['retweet_count'] = twitter_archive_cleaned['re_fav_reply'].apply(lambda x: x[1:-1].split(',')[0][1:-1])
twitter_archive_cleaned['favorite_count'] = twitter_archive_cleaned['re_fav_reply'].apply(lambda x: x[1:-1].split(',')[1][2:-1])
twitter_archive_cleaned['reply_count'] = twitter_archive_cleaned['re_fav_reply'].apply(lambda x: x[1:-1].split(',')[2][2:-1])
twitter_archive_cleaned = twitter_archive_cleaned.drop('re_fav_reply', 1)

In [44]:
twitter_archive_cleaned['retweet_count'].value_counts()

        10
129      6
305      5
587      5
1127     5
492      4
1155     4
389      4
1340     4
414      4
328      4
585      4
58       4
244      4
154      4
371      4
1295     4
2        3
298      3
555      3
870      3
1840     3
582      3
954      3
1902     3
542      3
3600     3
6296     3
663      3
421      3
        ..
748      1
3023     1
262      1
3008     1
5574     1
3442     1
890      1
2833     1
1524     1
728      1
1917     1
1624     1
794      1
1647     1
130      1
544      1
2456     1
1705     1
911      1
995      1
8532     1
4706     1
741      1
378      1
4636     1
56       1
128      1
1243     1
2412     1
2680     1
Name: retweet_count, Length: 1740, dtype: int64

In [45]:
twitter_archive_cleaned['favorite_count'].value_counts()

         10
761       3
3472      3
3838      3
5400      3
2311      3
691       3
19552     3
2392      3
1370      3
2814      2
3303      2
4735      2
476       2
35237     2
1743      2
148       2
3215      2
7095      2
2093      2
2999      2
2628      2
4678      2
1978      2
2950      2
3133      2
17292     2
21649     2
181       2
1561      2
         ..
5194      1
3419      1
19808     1
6209      1
3514      1
256       1
14925     1
16303     1
2735      1
13394     1
1367      1
7128      1
17839     1
17391     1
1172      1
9434      1
18342     1
3409      1
858       1
1281      1
11189     1
53332     1
108       1
1896      1
930       1
13688     1
7802      1
1497      1
4949      1
3500      1
Name: favorite_count, Length: 2019, dtype: int64

In [46]:
twitter_archive_cleaned['reply_count'].value_counts()

2       49
15      46
3       43
18      43
4       41
16      41
13      40
27      39
33      39
6       38
11      38
9       37
10      36
14      36
29      35
5       34
38      34
28      33
25      33
22      33
21      33
8       32
20      32
34      31
1       30
23      30
19      30
26      30
31      30
17      30
        ..
235      1
243      1
659      1
814      1
2070     1
588      1
350      1
655      1
601      1
186      1
125      1
178      1
461      1
376      1
232      1
200      1
170      1
110      1
228      1
230      1
129      1
202      1
216      1
381      1
164      1
1008     1
244      1
176      1
422      1
184      1
Name: reply_count, Length: 252, dtype: int64

In [47]:
twitter_archive_cleaned

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,breed_name,stage,retweet_count,favorite_count,reply_count
0,666020888022790149,0,0,2015-11-15 22:32:08,Twitter for iPhone,Here we have a Japanese Irish Setter. Lost eye...,0,0,NaN,https://twitter.com/dog_rates/status/666020888...,8.0,10,None,welsh_springer_spaniel,NaN,515,2562,25
1,666029285002620928,0,0,2015-11-15 23:05:30,Twitter for iPhone,This is a western brown Mitsubishi terrier. Up...,0,0,NaN,https://twitter.com/dog_rates/status/666029285...,7.0,10,None,redbone,NaN,47,130,0
2,666033412701032449,0,0,2015-11-15 23:21:54,Twitter for iPhone,Here is a very happy pup. Big fan of well-main...,0,0,NaN,https://twitter.com/dog_rates/status/666033412...,9.0,10,None,german_shepherd,NaN,44,125,1
3,666044226329800704,0,0,2015-11-16 00:04:52,Twitter for iPhone,This is a purebred Piers Morgan. Loves to Netf...,0,0,NaN,https://twitter.com/dog_rates/status/666044226...,6.0,10,None,rhodesian_ridgeback,NaN,141,299,1
4,666049248165822465,0,0,2015-11-16 00:24:50,Twitter for iPhone,Here we have a 1949 1st generation vulpix. Enj...,0,0,NaN,https://twitter.com/dog_rates/status/666049248...,5.0,10,None,miniature_pinscher,NaN,40,108,7
5,666050758794694657,0,0,2015-11-16 00:30:50,Twitter for iPhone,This is a truly beautiful English Wilson Staff...,0,0,NaN,https://twitter.com/dog_rates/status/666050758...,10.0,10,None,bernese_mountain_dog,NaN,58,133,0
6,666051853826850816,0,0,2015-11-16 00:35:11,Twitter for iPhone,This is an odd dog. Hard on the outside but lo...,0,0,NaN,https://twitter.com/dog_rates/status/666051853...,2.0,10,None,NaN,NaN,848,1217,11
7,666055525042405380,0,0,2015-11-16 00:49:46,Twitter for iPhone,Here is a Siberian heavily armored polar bear ...,0,0,NaN,https://twitter.com/dog_rates/status/666055525...,10.0,10,None,chow,NaN,249,434,2
8,666057090499244032,0,0,2015-11-16 00:55:59,Twitter for iPhone,My oh my. This is a rare blond Canadian terrie...,0,0,NaN,https://twitter.com/dog_rates/status/666057090...,9.0,10,None,NaN,NaN,141,296,2
9,666058600524156928,0,0,2015-11-16 01:01:59,Twitter for iPhone,Here is the Rand Paul of retrievers folks! He'...,0,0,NaN,https://twitter.com/dog_rates/status/666058600...,8.0,10,None,miniature_poodle,NaN,57,111,4


##### It seems like to_csv and read_csv changes the datatype. Let us leave this as is and take care of same in visualization.

In [48]:
twitter_archive_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2210 entries, 0 to 2209
Data columns (total 18 columns):
tweet_id                      2210 non-null int64
in_reply_to_status_id         2210 non-null int64
in_reply_to_user_id           2210 non-null int64
timestamp                     2210 non-null object
source                        2210 non-null object
text                          2210 non-null object
retweeted_status_id           2210 non-null int64
retweeted_status_user_id      2210 non-null int64
retweeted_status_timestamp    69 non-null object
expanded_urls                 2169 non-null object
rating_numerator              2210 non-null float64
rating_denominator            2210 non-null int64
name                          2210 non-null object
breed_name                    1512 non-null object
stage                         426 non-null object
retweet_count                 2210 non-null object
favorite_count                2210 non-null object
reply_count                   2210

In [49]:
twitter_archive_cleaned.to_csv("twitter_archive_master.csv")

#### We are done with cleaning......Let us move on for visualization :)